In [1]:
!pip install keras

In [2]:
import numpy as np
import pandas as pd

In [ ]:
!pip install -q konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/skku/인공지능앱개발대회/sample3.csv')

In [5]:
df.head(1)

,Unnamed: 0,no,new,title,link,text,date,visited
0,0,13256.0,None,프로지니어㈜ 연구원 채용,?mode=view&articleNo=90433&article.offset=0&ar...,◎ 모집분야: \r\n1. 신약개발 연구원\r\n담당업무:\r\n■ 분자생물학 관련...,2021-06-20,1009.0


In [6]:
dfp = pd.read_csv('/content/drive/MyDrive/skku/인공지능앱개발대회/notice_preprocess.csv')

In [7]:
dfp.head(5)

,title,text,body
0,프로지니어㈜ 연구원 채용,◎ 모집분야: \r\n1. 신약개발 연구원\r\n담당업무:\r\n■ 분자생물학 관련...,모집분야 1 신약개발 연구원 담당업무 분자생물학 관련 실험 업무 혈액 조직에서 추출...
1,"독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 “시민참여와 ...","독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 \r\n\r...",독일프리드리히나우만재단 미래인문학기반사회혁신창업교육연구단 공동 주최 시민참여와 사회...
2,[응용통계연구소] 2021년 하계 통계특강 개최(온라인 진행),<접수기간 및 접수방법>\r\n\r\n\r\n- R 기초 강좌 수강료 (등록비 1만...,접수기간 및 접수방법 기초 강좌 수강료 등록비 1만원 및 교재포함 강좌별별도 학생 ...
3,[자연과학캠퍼스] 우편취급국(수원) 근로학생(졸업생 포함) 모집 (기간연장),성균관대학교 자연과학캠퍼스 우편취급국(수원 소재)에서 사무보조 업무를 수행할 근로학...,성균관대학교 자연과학캠퍼스 우편취급국 수원 소재 에서 사무보조 업무를 수행할 근로학...
4,SW융합대학 글로벌융합학부 자체직원 모집 공고,우리 성균관대학교 SW융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 ...,우리 성균관대학교 융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 많은...


In [8]:
dfp_train = dfp[20:]
dfp_test = dfp[:20]

In [9]:
from konlpy.tag import Mecab

mecab = Mecab()

def get_tokens(x):
    try:
        return [i for i in mecab.nouns(x) if len(i) > 1] if x else []
    except Exception as e:
        if str(x) == 'nan':
            return []
        print(e)
        print(str(x))
        raise e

In [10]:
dfp_train['mecab'] = dfp_train['body'].map(get_tokens)
dfp_train['mecab_len'] = dfp_train['mecab'].map(len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
dfp_train = dfp_train[dfp_train['mecab_len'] >= 5]
dfp_train

,title,text,body,mecab,mecab_len
20,2021-2022 이탈리아정부초청 장학생 선발 모집,2021-2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,2021 2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,"[이탈리아, 정부, 초청, 장학생, 모집, 학생, 관심, 부탁, 선발, 과정, 장학...",29
21,[창업지원단 캠퍼스타운사업단] 종로청년주거활동프로젝트 참여자 모집,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,"[창업, 지원, 캠퍼스, 타운, 사업단, 주거, 취약, 계층, 청년, 주거, 정책,...",202
22,[양자정보연구지원센터] 2021년 2차 신진연구인력 연수프로그램 연수생 모집,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,"[과학, 기술, 정보, 통신부, 지정, 양자, 정보, 연구, 지원, 센터, 양자, ...",99
23,성균관대학교 SKK GSB행정실 행정조교 모집,성균관대학교 SKK Graduate School of Business(이하 SKK ...,성균관대학교 이하 는 영국 파이낸셜타임즈 선정 10년 연속 한국 1위 세계 35위 ...,"[성균관, 대학교, 영국, 파이낸셜, 타임즈, 선정, 연속, 한국, 세계, 스쿨, ...",170
24,수료 후 6개월 초과자 3품(삼품) 제출 면제신청 기한 연장 안내,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,"[수료, 개월, 초과, 미취, 수료자, 면제, 관련, 아래, 신청, 자격, 수료, ...",47
...,...,...,...,...,...
13251,청년창업 토크콘서트,"\r\n청년창업토크콘서트 ""창업그냥한번해봐"" 에 많은 참석을 바랍니다.\r\n새로...",청년창업토크콘서트 창업그냥한번해봐 에 많은 참석을 바랍니다 새로운 시장지배 세력으로...,"[청년, 창업, 토크, 콘서트, 창업, 참석, 시장, 지배, 세력, 등장, 스마트,...",34
13252,[봉룡학사]체험형 힐링 성악강좌 개최,세종예술아카데미의 '히든 보이스'라는 강좌에 대해 들어본 적 있으신가요?\r\n\r...,세종예술아카데미의 히든 보이스 라는 강좌에 대해 들어본 적 있으신가요 히든 보이스 ...,"[세종, 예술, 아카데미, 히든, 보이스, 강좌, 가요, 히든, 보이스, 세종문화회...",105
13254,공인어학시험 최초응시자 할인안내,2014학년도 1학기에 국제품 취득 혹은 어학능력향상을 위해 공인어학시험에 응시하는...,2014학년도 1학기에 국제품 취득 혹은 어학능력향상을 위해 공인어학시험에 응시하는...,"[학년도, 학기, 국제, 취득, 어학, 능력, 향상, 공인, 어학, 시험, 응시, ...",21
13255,2014 공과대학 학부모초청간담회 안내,"2014학년도 공과대학 각 전공에 진입한 2학년, 3학년, 4학년 학생들의 학부모님...",2014학년도 공과대학 각 전공에 진입한 2학년 3학년 4학년 학생들의 학부모님들을...,"[학년도, 공과, 대학, 전공, 진입, 학년, 학년, 학년, 학생, 학부모, 자녀,...",79


In [12]:
from gensim.models import Word2Vec
model = Word2Vec(dfp_train['mecab'], size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

In [13]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(model.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0)
    except IndexError as e:
        pass

In [14]:
dfp_train['w2v'] = dfp_train['mecab'].map(get_sentence_mean_vector)

In [15]:
dfp_train.head()

,title,text,body,mecab,mecab_len,w2v
20,2021-2022 이탈리아정부초청 장학생 선발 모집,2021-2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,2021 2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,"[이탈리아, 정부, 초청, 장학생, 모집, 학생, 관심, 부탁, 선발, 과정, 장학...",29,"[-0.27521124, 0.077782646, -0.049963262, -0.11..."
21,[창업지원단 캠퍼스타운사업단] 종로청년주거활동프로젝트 참여자 모집,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,"[창업, 지원, 캠퍼스, 타운, 사업단, 주거, 취약, 계층, 청년, 주거, 정책,...",202,"[-0.1923414, 0.08821088, -0.0011745733, -0.126..."
22,[양자정보연구지원센터] 2021년 2차 신진연구인력 연수프로그램 연수생 모집,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,"[과학, 기술, 정보, 통신부, 지정, 양자, 정보, 연구, 지원, 센터, 양자, ...",99,"[-0.18443398, -0.05779633, 0.032880418, -0.202..."
23,성균관대학교 SKK GSB행정실 행정조교 모집,성균관대학교 SKK Graduate School of Business(이하 SKK ...,성균관대학교 이하 는 영국 파이낸셜타임즈 선정 10년 연속 한국 1위 세계 35위 ...,"[성균관, 대학교, 영국, 파이낸셜, 타임즈, 선정, 연속, 한국, 세계, 스쿨, ...",170,"[-0.34534433, -0.07878227, -0.10842035, -0.294..."
24,수료 후 6개월 초과자 3품(삼품) 제출 면제신청 기한 연장 안내,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,"[수료, 개월, 초과, 미취, 수료자, 면제, 관련, 아래, 신청, 자격, 수료, ...",47,"[-0.20343798, 0.060435157, -0.1807757, -0.1448..."


In [16]:
from sklearn.cluster import KMeans
import time

word_vectors = dfp_train.w2v.to_list() 
num_clusters = 25

# K-means
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)
dfp_train['category'] = idx

In [17]:
dfp_train

,title,text,body,mecab,mecab_len,w2v,category
20,2021-2022 이탈리아정부초청 장학생 선발 모집,2021-2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,2021 2022 이탈리아정부초청 장학생을 모집 하오니 학생들의 많은 관심 부탁드립...,"[이탈리아, 정부, 초청, 장학생, 모집, 학생, 관심, 부탁, 선발, 과정, 장학...",29,"[-0.27521124, 0.077782646, -0.049963262, -0.11...",24
21,[창업지원단 캠퍼스타운사업단] 종로청년주거활동프로젝트 참여자 모집,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,창업지원단 캠퍼스타운사업단에서는 주거 취약계층인 청년에게 주거정책 이론 및 실무교육...,"[창업, 지원, 캠퍼스, 타운, 사업단, 주거, 취약, 계층, 청년, 주거, 정책,...",202,"[-0.1923414, 0.08821088, -0.0011745733, -0.126...",8
22,[양자정보연구지원센터] 2021년 2차 신진연구인력 연수프로그램 연수생 모집,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,과학기술정보통신부 지정 양자정보연구지원센터는 양자정보과학 분야 인적 기반 조성을 위...,"[과학, 기술, 정보, 통신부, 지정, 양자, 정보, 연구, 지원, 센터, 양자, ...",99,"[-0.18443398, -0.05779633, 0.032880418, -0.202...",22
23,성균관대학교 SKK GSB행정실 행정조교 모집,성균관대학교 SKK Graduate School of Business(이하 SKK ...,성균관대학교 이하 는 영국 파이낸셜타임즈 선정 10년 연속 한국 1위 세계 35위 ...,"[성균관, 대학교, 영국, 파이낸셜, 타임즈, 선정, 연속, 한국, 세계, 스쿨, ...",170,"[-0.34534433, -0.07878227, -0.10842035, -0.294...",6
24,수료 후 6개월 초과자 3품(삼품) 제출 면제신청 기한 연장 안내,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,수료 후 6개월 초과 3품 미취득 수료자 3품 면제와 관련해 아래와 같이 안내드립니...,"[수료, 개월, 초과, 미취, 수료자, 면제, 관련, 아래, 신청, 자격, 수료, ...",47,"[-0.20343798, 0.060435157, -0.1807757, -0.1448...",10
...,...,...,...,...,...,...,...
13251,청년창업 토크콘서트,"\r\n청년창업토크콘서트 ""창업그냥한번해봐"" 에 많은 참석을 바랍니다.\r\n새로...",청년창업토크콘서트 창업그냥한번해봐 에 많은 참석을 바랍니다 새로운 시장지배 세력으로...,"[청년, 창업, 토크, 콘서트, 창업, 참석, 시장, 지배, 세력, 등장, 스마트,...",34,"[-0.003697354, 0.064199455, -0.1254645, -0.288...",3
13252,[봉룡학사]체험형 힐링 성악강좌 개최,세종예술아카데미의 '히든 보이스'라는 강좌에 대해 들어본 적 있으신가요?\r\n\r...,세종예술아카데미의 히든 보이스 라는 강좌에 대해 들어본 적 있으신가요 히든 보이스 ...,"[세종, 예술, 아카데미, 히든, 보이스, 강좌, 가요, 히든, 보이스, 세종문화회...",105,"[-0.32373744, 0.20351286, -0.14917716, -0.2807...",1
13254,공인어학시험 최초응시자 할인안내,2014학년도 1학기에 국제품 취득 혹은 어학능력향상을 위해 공인어학시험에 응시하는...,2014학년도 1학기에 국제품 취득 혹은 어학능력향상을 위해 공인어학시험에 응시하는...,"[학년도, 학기, 국제, 취득, 어학, 능력, 향상, 공인, 어학, 시험, 응시, ...",21,"[-0.3707478, -0.07407768, -0.17255268, -0.2656...",9
13255,2014 공과대학 학부모초청간담회 안내,"2014학년도 공과대학 각 전공에 진입한 2학년, 3학년, 4학년 학생들의 학부모님...",2014학년도 공과대학 각 전공에 진입한 2학년 3학년 4학년 학생들의 학부모님들을...,"[학년도, 공과, 대학, 전공, 진입, 학년, 학년, 학년, 학생, 학부모, 자녀,...",79,"[-0.20548691, -0.01676282, 0.049047034, -0.434...",20


In [18]:
dfp_train.category.value_counts()

6     1530
1     1044
24    1026
4      880
7      764
19     748
20     734
8      688
10     560
21     554
22     460
18     459
11     432
23     418
3      373
5      353
9      352
2      350
12     265
0      246
15     229
14     156
17      58
16      52
13      42
Name: category, dtype: int64

## 각 카테고리별 키워드 정리

In [19]:
! pip install krwordrank
from krwordrank.word import KRWordRank

In [20]:
dfp_train = dfp_train.dropna()

In [21]:
wordrank_extractor = KRWordRank(
    min_count = 5,
    max_length = 10, 
    verbose = True
    )

beta = 0.85  
max_iter = 10

In [22]:
dfp_test['mecab'] = dfp_test['body'].map(get_tokens)
dfp_test['mecab_len'] = dfp_test['mecab'].map(len)
dfp_test = dfp_test[dfp_test['mecab_len'] >= 5]
dfp_test['w2v'] = dfp_test['mecab'].map(get_sentence_mean_vector)
word_vectors_test = dfp_test.w2v.to_list()

test_idx = kmeans_clustering.predict(word_vectors_test)
dfp_test['category'] = test_idx

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
dfp_test

,title,text,body,mecab,mecab_len,w2v,category
0,프로지니어㈜ 연구원 채용,◎ 모집분야: \r\n1. 신약개발 연구원\r\n담당업무:\r\n■ 분자생물학 관련...,모집분야 1 신약개발 연구원 담당업무 분자생물학 관련 실험 업무 혈액 조직에서 추출...,"[모집, 분야, 신약, 개발, 연구원, 담당, 업무, 분자, 생물학, 관련, 실험,...",221,"[-0.38503844, -0.077162795, -0.08184796, -0.27...",22
1,"독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 “시민참여와 ...","독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 \r\n\r...",독일프리드리히나우만재단 미래인문학기반사회혁신창업교육연구단 공동 주최 시민참여와 사회...,"[독일, 프리드리히, 나우만, 재단, 미래, 인문학, 기반, 사회, 혁신, 창업, ...",131,"[-0.07585526, 0.15605238, 0.15880494, -0.29815...",19
2,[응용통계연구소] 2021년 하계 통계특강 개최(온라인 진행),<접수기간 및 접수방법>\r\n\r\n\r\n- R 기초 강좌 수강료 (등록비 1만...,접수기간 및 접수방법 기초 강좌 수강료 등록비 1만원 및 교재포함 강좌별별도 학생 ...,"[접수, 기간, 접수, 방법, 기초, 강좌, 수강료, 등록비, 교재, 포함, 강좌,...",100,"[-0.3741582, 0.019431533, -0.41744062, -0.1832...",10
3,[자연과학캠퍼스] 우편취급국(수원) 근로학생(졸업생 포함) 모집 (기간연장),성균관대학교 자연과학캠퍼스 우편취급국(수원 소재)에서 사무보조 업무를 수행할 근로학...,성균관대학교 자연과학캠퍼스 우편취급국 수원 소재 에서 사무보조 업무를 수행할 근로학...,"[성균관, 대학교, 자연, 과학, 캠퍼스, 우편, 취급, 수원, 소재, 사무, 보조...",112,"[-0.34426203, -0.18098561, -0.12864451, -0.353...",6
4,SW융합대학 글로벌융합학부 자체직원 모집 공고,우리 성균관대학교 SW융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 ...,우리 성균관대학교 융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 많은...,"[우리, 성균관, 대학교, 융합, 대학, 글로벌, 융합, 학부, 아래, 자체, 직원...",203,"[-0.3524098, -0.0360677, 0.015457532, -0.33045...",6
5,[바이오메디컬글로벌인재양성사업단] 보건복지부 지원 해외파견 대학원생 연구자 모집,바이오메디컬 글로벌 인재 양성사업\r\n- 2021년 3차 해외파견 연구자 모집 공...,바이오메디컬 글로벌 인재 양성사업 2021년 3차 해외파견 연구자 모집 공고 개요 ...,"[바이오, 메디컬, 글로벌, 인재, 양성, 사업, 해외, 파견, 연구자, 모집, 공...",230,"[-0.30125868, -0.08129884, 0.006400729, -0.168...",22
6,[LINC+]ESG Venture 크라우드 펀딩 교육 참가자 모집(재공지),"1. 교육을 통해 얻을 수 있는것!\r\n- 벤처 창업, ESG, 크라우드펀딩에 대...",1 교육을 통해 얻을 수 있는것 벤처 창업 크라우드펀딩에 대한 이론적인 이해 팀 프...,"[교육, 벤처, 창업, 우드, 펀딩, 이론, 이해, 프로젝트, 기반, 제품, 서비스...",88,"[-0.19010416, 0.09402438, -0.10444059, -0.2749...",1
7,[LINC+] '랜선으로 만나는 릴레이 글로벌 취창업 톡톡톡' 참가 안내,LINC+사업단에서는 코로나19 영향으로 글로벌 취업·창업· 인턴십 등에 어려움을 ...,사업단에서는 코로나19 영향으로 글로벌 취업 창업 인턴십 등에 어려움을 겪고 있는 ...,"[사업단, 코로나, 영향, 글로벌, 취업, 창업, 인턴, 어려움, 재학, 해외, 연...",25,"[-0.19420399, 0.1899721, 0.054392535, -0.19418...",3
8,[LINC+](창의품) 창며드는 여름방학! '대학연합 B대면 창업 데이트' 참가 안내,LINC+사업단에서는 아주대·한양대 ERICA·부산대·포항공대 재학생들과 함께하는 ...,사업단에서는 아주대 한양대 부산대 포항공대 재학생들과 함께하는 제3회 대학연합창업캠...,"[사업단, 한양대, 부산대, 포항, 공대, 재학, 대학, 연합, 창업, 캠프, 창업...",82,"[-0.21700199, 0.1213668, -0.10786768, -0.25016...",1
9,인사캠 FGI참가자 모집공고,인사캠 FGI참가자 모집공고\r\n\r\n성균관대학교 콘텐츠 랩에서 진행하는 ‘미디...,인사캠 참가자 모집공고 성균관대학교 콘텐츠 랩에서 진행하는 미디어 신뢰 조사 의 참...,"[인사, 참가자, 모집, 공고, 성균관, 대학교, 콘텐츠, 진행, 미디어, 신뢰, ...",85,"[-0.24185063, 0.008599812, -0.19386138, -0.266...",1


In [24]:
word_list = []
for i in range(num_clusters):
    tmp= []
    try:
        keywords, rank, graph = wordrank_extractor.extract(dfp_train[dfp_train['category'] == i].body.tolist(), beta, max_iter)
    except AttributeError:
        keywords, rank, graph = wordrank_extractor.extract(dfp_train[dfp_train['category'] == i].body.tolist(), beta, max_iter)
        for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
            try:
                word = int(word)
            except ValueError:
                tmp.append(word)
    word_list.append(tmp)

scan vocabs ... 
num vocabs = 657
done
scan vocabs ... 
num vocabs = 13843
done
scan vocabs ... 
num vocabs = 7846
done
scan vocabs ... 
num vocabs = 6054
done
scan vocabs ... 
num vocabs = 15415
done
scan vocabs ... 
num vocabs = 6241
done
scan vocabs ... 
num vocabs = 10329
done
scan vocabs ... 
num vocabs = 6291
done
scan vocabs ... 
num vocabs = 11103
done
scan vocabs ... 
num vocabs = 4904
done
scan vocabs ... 
num vocabs = 9516
done
scan vocabs ... 
num vocabs = 6439
done
scan vocabs ... 
num vocabs = 5150
done
scan vocabs ... 
num vocabs = 262
done
scan vocabs ... 
num vocabs = 1659
done
scan vocabs ... 
num vocabs = 4005
done
scan vocabs ... 
num vocabs = 628
done
scan vocabs ... 
num vocabs = 261
done
scan vocabs ... 
num vocabs = 10310
done
scan vocabs ... 
num vocabs = 12112
done
scan vocabs ... 
num vocabs = 6910
done
scan vocabs ... 
num vocabs = 9891
done
scan vocabs ... 
num vocabs = 9132
done
scan vocabs ... 
num vocabs = 6965
done
scan vocabs ... 
num vocabs = 13545
do

In [25]:
for words in word_list:
    print(words)

[]
['신청', '프로그램', '참여', '학생', '바랍니다', '많은', '교육', '문의', '성균', '있는', '진행', '대학', '위한', '통해', '온라인', '지원', '홈페이지', '있습니다', '안내', '가능', '장소', '안녕하세요', '대상', '같이', '참가', '모집']
['신청', '학생', '바랍니다', '전공', '학과', '이수', '경우', '2학기', '1학기', '제출', '지원', '복수', '현장', '안내', '가능', '이상', '재학생', '같이', '참여', '작성', '모집', '있습니다', '기간', '참고', '실습', '진입', '교과목']
['창업', '지원', '신청', '사업', '성균관대', '모집', '많은', '참여', '프로그램', '바랍니다', '또는', '제출', '접수', '캠퍼스타운', '위한', '교육', '참가', '있는', '문의', '예비', '같이', '아이디어', '관심', '가능', '청년']
['지원', '제출', '장학', '선발', '신청', '1부', '또는', '대학', '경우', '접수', '홈페이지', '학생', '바랍니다', '서류', '이상', '성적', '같이', '재학', '많은', '문의', '2학기', '1학기', '가능', '지급', '반드시', '추천', '있는', '기준']
['바랍니다', '사용', '신청', '경우', '촬영', '대여', '인사캠', '운동장', '같이', '사전', '학생', '예약', '자과캠', '가능', '주시기', '문의', '안내', '1층', '단체', '따라', '학위복', '시간', '아래', '이용', '진행', '1학기', '학교']
['근무', '지원', '제출', '본교', '채용', '1부', '접수', '서류', '행정', '같이', '이상', '성균', '면접', '문의', '모집', '한해', '아래와', '개별', '대학', '업무', '우대', '경우', '함께', '연구', '가

## 키워드 입력 시 인공지능 추천 공지사항

In [26]:
keyword = ['인공지능', '장학금', '채용', '근로', '취업', '경영학']

In [27]:
max_score = []
total = []
for k in keyword: 
    score = [0 for _ in range(len(word_list))]
    for i, words in enumerate(word_list):
        for w in words:
            try:
                sim = model.wv.similarity(k, w)
            except KeyError:
                pass
            score[i] += sim

    print(k, max(score), score)
    max_score.append(max(score))
    total.append(score)

인공지능 4.9594715386629105 [0, 3.538209691643715, 3.588352955877781, 3.4649974331259727, 1.8729991782456636, 2.971898417919874, 2.65491240657866, 1.7219102829694748, 1.9240018539130688, 1.7565102463122457, 1.6123851267620921, 3.5202771686017513, 2.305015319958329, 4.683997645974159, 2.848373331129551, 2.8636134900152683, 2.037444744259119, 2.292917830636725, 2.0956130335107446, 4.089905433356762, 3.947961062192917, 3.4184087365865707, 3.706174161285162, 4.9594715386629105, 2.3564797630533576]
장학금 9.484346821904182 [0, 3.7918621404096484, 4.741766920313239, 3.7508178730495274, 7.5180816277861595, 3.6583291310817003, 4.073725625872612, 4.478844569995999, 6.868515368551016, 3.2527195662260056, 5.111069744452834, 3.615342970006168, 4.676655314862728, 2.3979043206200004, 3.57594296336174, 8.913008376955986, 3.7866133456118405, 2.1781088765710592, 9.484346821904182, 3.0398521148599684, 2.1760893473401666, 4.770017132163048, 4.643062278628349, 2.7492061434313655, 4.596325539052486]
채용 6.72194658

In [28]:
index = 2 #인공지능2 장학금19 분자생물학14(...) 근로10 취업9(카운슬링...) 경영학2

In [29]:
sim_idx = total[index].index(max_score[index]) #인공지능
dfp_train[dfp_train['category'] == sim_idx].head(20)

,title,text,body,mecab,mecab_len,w2v,category
23,성균관대학교 SKK GSB행정실 행정조교 모집,성균관대학교 SKK Graduate School of Business(이하 SKK ...,성균관대학교 이하 는 영국 파이낸셜타임즈 선정 10년 연속 한국 1위 세계 35위 ...,"[성균관, 대학교, 영국, 파이낸셜, 타임즈, 선정, 연속, 한국, 세계, 스쿨, ...",170,"[-0.34534433, -0.07878227, -0.10842035, -0.294...",6
39,"사회과학대학 소비자학과 사무실 행정조교 모집(~6/30, 수)",성균관대학교 사회과학대학 소비자학과 사무실에서 행정업무를 담당할 행정조교를 다음과 ...,성균관대학교 사회과학대학 소비자학과 사무실에서 행정업무를 담당할 행정조교를 다음과 ...,"[성균관, 대학교, 사회, 과학, 대학, 소비, 자학, 사무실, 행정, 업무, 담당...",179,"[-0.29273432, -0.11551516, -0.07417192, -0.342...",6
42,한국철학문화연구소 전문연구인력 채용,"성균관대학교 한국철학문화연구소에서는 한국철학을 세계에 효과적으로 발신, 확산하기 위...",성균관대학교 한국철학문화연구소에서는 한국철학을 세계에 효과적으로 발신 확산하기 위한...,"[성균관, 대학교, 한국, 철학, 문화, 연구소, 한국, 철학, 세계, 효과, 발신...",236,"[-0.30565622, -0.1192618, -0.10116877, -0.2395...",6
45,[사범대학] 교원양성대학 사업관리 연구원(팀장급) 채용(~7.7까지),성균관대학교 사범대학 사업관리 연구원(팀장급) 채용 공고\r\n\r\n성균관대학교 ...,성균관대학교 사범대학 사업관리 연구원 팀장급 채용 공고 성균관대학교 사범대학 사업관...,"[성균관, 대학교, 사범, 대학, 사업, 관리, 연구원, 팀장, 채용, 공고, 성균...",264,"[-0.3031475, -0.03519001, -0.066485815, -0.279...",6
47,[성균관대학교 BT강소기업상생지원센터] 자체계약직원 채용 공고,성균관대학교 BT강소기업상생지원센터 자체계약직원 채용 공고\r\n\r\n\r\n성균...,성균관대학교 강소기업상생지원센터 자체계약직원 채용 공고 성균관대학교 강소기업 상생지...,"[성균관, 대학교, 강소, 기업, 상생, 지원, 센터, 자체, 계약, 직원, 채용,...",226,"[-0.25961208, -0.04346477, -0.11183041, -0.317...",6
53,성균관대학교 산학교수(비전임교원) 채용 안내,성균관대학교 산학협력단에서는 산업통상자원부 「복합재료 동시설계 산업기술 거점센터」과...,성균관대학교 산학협력단에서는 산업통상자원부 복합재료 동시설계 산업기술 거점센터 과제...,"[성균관, 대학교, 산학, 협력, 산업, 통상, 자원, 복합, 재료, 동시, 설계,...",185,"[-0.35544756, -0.10984108, -0.062192984, -0.28...",6
57,성균관대학교 SKK GSB행정실 계약직원(AI MBA 과정 운영 담당) 모집 공고,성균관대학교 SKK Graduate School of Business(이하 SKK ...,성균관대학교 이하 는 영국 파이낸셜타임즈 선정 10년 연속 한국 1위 세계 35위 ...,"[성균관, 대학교, 영국, 파이낸셜, 타임즈, 선정, 연속, 한국, 세계, 스쿨, ...",158,"[-0.3837017, -0.090402, -0.13726902, -0.236397...",6
59,[㈜큐로젠 공고] 일반행정직 채용,"성균관대학교 자회사, 주식회사 큐로젠의 연구과제 행정업무를 담당할 일반행정직을 아래...",성균관대학교 자회사 주식회사 큐로젠의 연구과제 행정업무를 담당할 일반행정직을 아래와...,"[성균관, 대학교, 자회사, 주식회사, 연구, 과제, 행정, 업무, 담당, 일반, ...",110,"[-0.45007652, -0.17802086, -0.08344802, -0.177...",6
68,의과대학 분자종양면역학연구실 행정연구원 채용,성균관대학교 의과대학 분자종양면역학연구실에서 연구과제 행정업무를 담당할 행정연구원을...,성균관대학교 의과대학 분자종양면역학연구실에서 연구과제 행정업무를 담당할 행정연구원을...,"[성균관, 대학교, 의과, 대학, 분자, 종양, 면역학, 연구실, 연구, 과제, 행...",100,"[-0.42682156, -0.17119236, -0.10061128, -0.178...",6
75,성균관대학교 삼성융합의과학원행정실 자체계약직원 모집,성균관대학교 삼성융합의과학원에서 아래와 같이 자체계약직원을 채용하고자 하오니 많은 ...,성균관대학교 삼성융합의과학원에서 아래와 같이 자체계약직원을 채용하고자 하오니 많은 ...,"[성균관, 대학교, 삼성, 융합, 과학, 아래, 자체, 계약, 직원, 채용, 지원,...",148,"[-0.3639195, -0.10821103, -0.11923218, -0.2283...",6
